# Generalized Cellular Automata Test
The goal of this project is to experiment with a new paradigm for generalized intelligence, inspired heavily by [this](https://www.youtube.com/watch?v=p-OYPRhqRCg) talk between Joscha Bach and Michael Levin. The ideas I'm drawing upon are primarily Bachs, however I take heavy inspiration by Levins work on collective intelligence in all forms of cells as well, seeking originally to make one based mostly on the idea of bioelectric signaling, and slowly shifting more towards Joscha Bachs idea on a search for selector_functions to find appropraite neighbors in higher dimensions to process information over long distances.

In [3]:
import os
import imageio
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime
import csv
import ast
from matplotlib.patches import Patch
from matplotlib.colors import Normalize

In [23]:
# Activation Functions

def average_activation(neighbors):
    if neighbors:
        return sum(neighbor.state for neighbor in neighbors) / len(neighbors)
    return 0

def inverted_average_activation(neighbors):
    if neighbors:
        return 1 - (sum(neighbor.state for neighbor in neighbors) / len(neighbors))
    return 0

def sigmoid_activation(neighbors):
    total = sum(neighbor.state for neighbor in neighbors) if neighbors else 0
    return 1 / (1 + np.exp(-total))

def min_activation(neighbors):
    if neighbors:
        return min(neighbor.state for neighbor in neighbors)
    return 0

def max_activation(neighbors):
    if neighbors:
        return max(neighbor.state for neighbor in neighbors)
    return 0

def tanh_activation(neighbors):
    total = sum(neighbor.state for neighbor in neighbors) if neighbors else 0
    return (np.tanh(total) + 1) / 2  # Adjusting to [0, 1] range

def random_activation(neighbors):
    return (np.random.rand())

In [29]:
# Reward Functions

def reward_state(cell):
      return cell.state

def reward_inverted_state(cell):
      return 1 - cell.state

def reward_random(cell):
      return np.random.rand()

In [22]:
# Define your activation functions and their weights
activation_functions = [average_activation, inverted_average_activation, sigmoid_activation, min_activation, max_activation, tanh_activation, random_activation]
weights = [0.28, 0.2, 0.07, 0.3, 0.04, 0.07, 0.04]  # Adjust these weights as needed

# Define the get_color function with a fixed color map
def get_color(activation_function):
    color_map = {
        0: [255, 0, 0],    # Red | average_activation
        1: [0, 255, 0],      # Green | innverted_average_activation
        2: [0, 0, 255],    # Blue | sigmoid_activation
        3: [255, 255, 0],      # Yellow | min_activation
        4: [255, 0, 255],      # Magenta | max_activation
        5: [0, 255, 255],     # Cyan | tanh_activation
        6: [255, 140, 0]  # Orange | random_activation
    }
    return color_map.get(activation_function, [0, 0, 0])  # Default to black if not found

In [20]:
# Data & Visualization Tools

# Saves the state and activation functions of each cell given a grid, step, and save directory.
def save_grid_data_old(grid, step, save_dir):
    with open(f'{save_dir}/grid_data.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        for row in grid:
            writer.writerow([step] + [[cell.state, activation_functions.index(cell.activation_function)] for cell in row]) 

def save_grid_data(grid, step, save_dir):
    with open(f'{save_dir}/grid_data.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        for row in grid:
            writer.writerow([step] + [[cell.state, activation_functions.index(cell.activation_function), str(cell.neighbors), str(cell.memory)] for cell in row])

def parse_cell_data(cell_data_str):
    """Safely parse the string representation of cell data into a Python list."""
    return ast.literal_eval(cell_data_str)

def get_color_with_intensity(color, intensity):
    """Interpolate between the given color and white based on intensity."""
    white = np.array([255, 255, 255])
    target_color = np.array(color)
    # Corrected interpolation formula
    return (target_color * intensity) + (white * (1 - intensity))


def create_images_from_csv_colored_with_intensity(save_dir):
    with open(f'{save_dir}/grid_data.csv', 'r') as file:
        reader = csv.reader(file)
        current_step = -1
        grid = []

        for row in reader:
            step = int(row[0])
            if step != current_step:
                if current_step != -1:
                    # Generate and save the previous step's image
                    grid_array = np.array([[get_color_with_intensity(get_color(cell_data[1]), cell_data[0]) for cell_data in row] for row in grid])
                    plt.imshow(grid_array / 255.0)  # Normalize color values
                    plt.title(f'Step {current_step}')
                    plt.savefig(f'{save_dir}/step_{current_step}_color_intensity.png')
                    plt.close()

                current_step = step
                grid = []

            grid_row = [parse_cell_data(cell_data_str) for cell_data_str in row[1:]]
            grid.append(grid_row)

        # Save the last step's image
        if grid:
            grid_array = np.array([[get_color_with_intensity(get_color(cell_data[1]), cell_data[0]) for cell_data in row] for row in grid]) / 255.0
            plt.imshow(grid_array)  # Normalize color values
            plt.title(f'Step {current_step}')
            plt.savefig(f'{save_dir}/step_{current_step}_color_intensity.png')
            plt.close()

def create_images_from_csv(save_dir):
    with open(f'{save_dir}/grid_data.csv', 'r') as file:
        reader = csv.reader(file)
        current_step = -1
        grid = []
        
        for row in reader:
            step = int(row[0])
            if step != current_step:
                if current_step != -1:
                    grid_array = np.array(grid)
                    total_activation = np.sum(grid_array)
                    
                    plt.imshow(grid_array, cmap='gray_r', norm=Normalize(vmin=0, vmax=1))
                    plt.title(f'Step {current_step}')
                    plt.text(x=0, y=-5, s=f'Total Activation: {total_activation:.2f}', fontsize=10, color='red')
                    plt.savefig(f'{save_dir}/step_{current_step}_gray.png')
                    plt.close()
                
                current_step = step
                grid = []

            # Use ast.literal_eval to safely evaluate the string representation of the list
            # Then extract the activation state (the first element of the list)
            grid_row = [ast.literal_eval(cell)[0] for cell in row[1:]]
            grid.append(grid_row)
        
        if grid:
            grid_array = np.array(grid)
            total_activation = np.sum(grid_array)
            
            plt.imshow(grid_array, cmap='gray_r', norm=Normalize(vmin=0, vmax=1))
            plt.title(f'Step {current_step}')
            plt.text(x=0, y=-5, s=f'Total Activation: {total_activation:.2f}', fontsize=10, color='red')
            plt.savefig(f'{save_dir}/step_{current_step}_gray.png')
            plt.close()

def create_images_from_csv_colored(save_dir):
    with open(f'{save_dir}/grid_data.csv', 'r') as file:
        reader = csv.reader(file)
        current_step = -1
        grid = []
        
        for row in reader:
            step = int(row[0])
            if step != current_step:
                if current_step != -1:
                    # Generate and save the previous step's image
                    grid_array = np.array([[get_color(cell_data[1]) for cell_data in row] for row in grid])
                    plt.imshow(grid_array / 255.0)  # Normalize color values
                    plt.title(f'Step {current_step}')
                    plt.savefig(f'{save_dir}/step_{current_step}_color.png')
                    plt.close()
                
                current_step = step
                grid = []

            grid_row = [parse_cell_data(cell_data_str) for cell_data_str in row[1:]]
            grid.append(grid_row)
        
        # Save the last step's image
        if grid:
            grid_array = np.array([[get_color(cell_data[1]) for cell_data in row] for row in grid]) / 255.0
            plt.imshow(grid_array)  # Normalize color values
            plt.title(f'Step {current_step}')
            plt.savefig(f'{save_dir}/step_{current_step}_color.png')
            plt.close()

def create_side_by_side_images_from_csv(save_dir):
    with open(f'{save_dir}/grid_data.csv', 'r') as file:
        reader = csv.reader(file)
        current_step = -1
        grid = []
        
        for row in reader:
            step = int(row[0])
            if step != current_step:
                if current_step != -1:
                    # Generate the previous step's side-by-side image
                    gray_grid_array = np.array([[parse_cell_data(cell)[0] for cell in row] for row in grid])
                    colored_grid_array = np.array([[get_color(parse_cell_data(cell)[1]) for cell in row] for row in grid]) / 255.0
                    
                    # Create a figure to hold both images
                    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
                    axs[0].imshow(gray_grid_array, cmap='gray_r', norm=Normalize(vmin=0, vmax=1))
                    axs[0].set_title('Grayscale')
                    axs[0].axis('off')  # Hide the axes
                    
                    axs[1].imshow(colored_grid_array)
                    axs[1].set_title('Colored')
                    axs[1].axis('off')  # Hide the axes
                    
                    plt.suptitle(f'Step {current_step}')
                    plt.savefig(f'{save_dir}/step_{current_step}_side_by_side.png')
                    plt.close()
                
                current_step = step
                grid = []

            grid_row = [cell_data_str for cell_data_str in row[1:]]
            grid.append(grid_row)
        
        # Save the last step's side-by-side image
        if grid:
            gray_grid_array = np.array([[parse_cell_data(cell)[0] for cell in row] for row in grid])
            colored_grid_array = np.array([[get_color(parse_cell_data(cell)[1]) for cell in row] for row in grid]) / 255.0
            
            # Create a figure to hold both images
            fig, axs = plt.subplots(1, 2, figsize=(10, 5))
            axs[0].imshow(gray_grid_array, cmap='gray', norm=Normalize(vmin=0, vmax=1))
            axs[0].set_title('Grayscale')
            axs[0].axis('off')  # Hide the axes
            
            axs[1].imshow(colored_grid_array)
            axs[1].set_title('Colored')
            axs[1].axis('off')  # Hide the axes
            
            plt.suptitle(f'Step {current_step}')
            plt.savefig(f'{save_dir}/step_{current_step}_side_by_side.png')
            plt.close()

def create_side_by_side_images_with_key_from_csv(save_dir):
    with open(f'{save_dir}/grid_data.csv', 'r') as file:
        reader = csv.reader(file)
        current_step = -1
        grid = []
        
        for row in reader:
            step = int(row[0])
            if step != current_step:
                if current_step != -1:
                    # Generate the previous step's side-by-side image
                    gray_grid_array = np.array([[parse_cell_data(cell)[0] for cell in row] for row in grid])
                    colored_grid_array = np.array([[get_color(parse_cell_data(cell)[1]) for cell in row] for row in grid]) / 255.0
                    
                    # Create a figure to hold both images and a legend
                    fig, axs = plt.subplots(1, 3, figsize=(15, 5), gridspec_kw={'width_ratios': [1, 1, 0.2]})
                    axs[0].imshow(gray_grid_array, cmap='gray', norm=Normalize(vmin=0, vmax=1))
                    axs[0].set_title('Grayscale')
                    axs[0].axis('off')  # Hide the axes
                    
                    axs[1].imshow(colored_grid_array)
                    axs[1].set_title('Colored')
                    axs[1].axis('off')  # Hide the axes
                    
                    # Create a color key legend for the colored image
                    legend_elements = [
                        Patch(facecolor=np.array(color)/255.0, edgecolor='r', label=func.__name__)
                        for func, color in zip(activation_functions, get_color(range(len(activation_functions))).values())
                    ]
                    axs[2].legend(handles=legend_elements, loc='upper left')
                    axs[2].axis('off')  # Hide the axes for the legend subplot
                    
                    plt.suptitle(f'Step {current_step}')
                    plt.savefig(f'{save_dir}/step_{current_step}_side_by_side.png')
                    plt.close()
                
                current_step = step
                grid = []

            grid_row = [cell_data_str for cell_data_str in row[1:]]
            grid.append(grid_row)
        
        # Save the last step's side-by-side image
        if grid:
            gray_grid_array = np.array([[parse_cell_data(cell)[0] for cell in row] for row in grid])
            colored_grid_array = np.array([[get_color(parse_cell_data(cell)[1]) for cell in row] for row in grid]) / 255.0
            
            # Create a figure to hold both images and a legend
            fig, axs = plt.subplots(1, 3, figsize=(15, 5), gridspec_kw={'width_ratios': [1, 1, 0.2]})
            axs[0].imshow(gray_grid_array, cmap='gray', norm=Normalize(vmin=0, vmax=1))
            axs[0].set_title('Grayscale')
            axs[0].axis('off')  # Hide the axes
            
            axs[1].imshow(colored_grid_array)
            axs[1].set_title('Colored')
            axs[1].axis('off')  # Hide the axes
            
            # Create a color key legend for the colored image
            legend_elements = [
                Patch(facecolor=np.array(color)/255.0, edgecolor='r', label=func.__name__)
                for func, color in zip(activation_functions, get_color(range(len(activation_functions))).values())
            ]
            axs[2].legend(handles=legend_elements, loc='upper left')
            axs[2].axis('off')  # Hide the axes for the legend subplot
            
            plt.suptitle(f'Step {current_step}')
            plt.savefig(f'{save_dir}/step_{current_step}_side_by_side.png')
            plt.close()


def create_gif(image_folder, gif_name):
    images = []

    def sort_key(file_name):
        numbers = re.findall(r'\d+', file_name)
        return int(numbers[0]) if numbers else 0

    file_names = sorted(os.listdir(image_folder), key=sort_key)

    for file_name in file_names:
        if file_name.endswith('.png'):
            file_path = os.path.join(image_folder, file_name)
            images.append(imageio.imread(file_path))

    imageio.mimsave(os.path.join(image_folder, gif_name), images, fps=10)

    for file_name in file_names:
            if file_name.endswith('.png'):
                file_path = os.path.join(image_folder, file_name)
                os.remove(file_path)

In [38]:
# Main Experiment

# Create a unique directory name based on the current date and time
current_time = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
experiment_dir = f'Experiments/Experiment_3/Experiment_{current_time}'
os.makedirs(experiment_dir, exist_ok=True)

class Cell:
    MAX_RECENT_MEMORIES = 10

    def __init__(self, x, y, activation_function, reward_function, num_neighbors=4):
        self.x = x
        self.y = y
        self.state = 0.0
        self.reward = 0.0
        self.memory = []
        self.neighbors = []
        self.num_neighbors = num_neighbors
        self.connection_strength = {neighbor: 0.5 for neighbor in self.neighbors}
        self.activation_function = activation_function
        self.reward_function = reward_function

    def update_selector(self, update_cells=None):
        # If no specific cells are provided, update all
        if update_cells is None:
            update_cells = range(self.num_neighbors)

        # Update the specified neighbors
        for i in update_cells:
            # Randomly select a new neighbor
            new_neighbor_x = np.random.randint(0, grid_size[0])
            new_neighbor_y = np.random.randint(0, grid_size[1])
            new_neighbor = (new_neighbor_x, new_neighbor_y)

            # Replace the old neighbor with the new one
            if i < len(self.neighbors):
                self.neighbors[i] = new_neighbor
            else:
                self.neighbors.append(new_neighbor)

            # Initialize connection strength for the new neighbor
            self.connection_strength[new_neighbor] = 0.5

    def update_state(self):
        new_state = cell.activation_function(neighbors)

        if new_state > 1:
            new_state = 1
        elif new_state < 0:
            new_state = 0

        self.state = new_state        
        new_reward = cell.reward_function(cell)

        self.reward = new_reward

        memory = [self.state, self.reward]

        # Add the new memory and ensure the memory list does not exceed 10 items
        self.memory.append(memory)
        if len(self.memory) > self.MAX_RECENT_MEMORIES:
            self.memory.pop(0)  # Remove the oldest memory to maintain the size limit

# Function to get neighboring cells based on selector
def get_neighbors(cell):
    neighbors = []
    for i in cell.neighbors:
        neighbors.append(ca_grid[i[0], i[1]])

    return neighbors

# Define the cell update logic for continuous values
def update_cell_state(cell, neighbors):
    cell.update_state()

# Learning mechanism
def apply_learning(cell):
    average_rewards = sum([m[1] for m in cell.memory]) / len(cell.memory)

    # Complete reshuffle
    if average_rewards < 0.2:
        chosen_function = np.random.choice(activation_functions, p=weights)
        cell.activation_function = chosen_function
        cell.update_selector()

    # Seek new neighbor from lowest reward neighbor
    if average_rewards > 0.15 and average_rewards < 0.8:  # Low average reward
        neighbors = get_neighbors(cell)

        # Find the neighbor with the lowest sum of states over the last 10 memories
        lowest_sum_neighbor = None
        lowest_sum = float('inf')

        for neighbor in neighbors:
            sum_states = sum([m[1] for m in neighbor.memory])  # Sum of rewards

            if sum_states < lowest_sum:
                lowest_sum = sum_states
                lowest_sum_neighbor = neighbor

        # If a neighbor with the lowest sum is found, update its selector in the current cell
        if lowest_sum_neighbor:
            index_of_neighbor = cell.neighbors.index((lowest_sum_neighbor.x, lowest_sum_neighbor.y))
            cell.update_selector(update_cells=[index_of_neighbor])
        
# Initialize the CA grid with Cell objects
grid_size = (20, 20)
ca_grid = np.empty(grid_size, dtype=object)
cycle_length = 5
for i in range(grid_size[0]):
    for j in range(grid_size[1]):
        chosen_function = np.random.choice(activation_functions, p=weights)
        ca_grid[i, j] = Cell(i, j, chosen_function, reward_inverted_state)
        ca_grid[i, j].state = np.random.rand()
        ca_grid[i, j].update_selector()

# Main simulation loop
simulation_steps = 500
for step in range(simulation_steps):
    # Update each cell based on rules
    for i in range(grid_size[0]):
        for j in range(grid_size[1]):
            cell = ca_grid[i, j]
            neighbors = get_neighbors(cell)
            update_cell_state(cell, neighbors)
            if step % cycle_length == 0:
                apply_learning(cell)
                print()
    print(step)
    
    # Save as image for visualization
    save_grid_data(ca_grid, step, experiment_dir)

















































































































































































































































































































































































































0
1
2
3
4
















































































































































































































































































































































































































5
6
7
8
9






















































































































































































In [32]:
create_images_from_csv(experiment_dir)
create_gif(experiment_dir, 'ca_simulation_grayscale.gif')

C:\Users\CYBER\AppData\Local\Temp\ipykernel_15136\889298356.py:262: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file_path))


In [37]:
create_side_by_side_images_from_csv(experiment_dir)
create_gif(experiment_dir, 'ca_simulation_overview.gif')

C:\Users\CYBER\AppData\Local\Temp\ipykernel_15136\889298356.py:262: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file_path))


In [31]:
create_images_from_csv_colored_with_intensity(experiment_dir)
create_gif(experiment_dir, 'ca_simulation_color_intensity.gif')

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

In [6]:
create_gif('Experiments/Experiment_2/Experiment_2023-12-01-02-32-16', 'ca_simulation.gif')

C:\Users\CYBER\AppData\Local\Temp\ipykernel_5928\3792796118.py:58: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file_path))


# Random ideas
* Now that we have the cellular automata engine, it's time to make it think. The basic idea is to have it's selector function decide which neighbors it looks for, probably up to the maximum of 8 it coulde have around it spatially. We can intialize them entirely randomly, or simply start with it's base neighbors.
* To get useful inputs and outputs, we need to have certain neurons represent inputs, and others represent outputs. For instance, we could encode some information in the continuous value of input neurons, with more neurons used for more complex information, and have the available actions mapped to certain outputs (ideally these outputs would be more arbitary and less hard-coded somehow in the future).
* The learning mechanism will be reward driven using an economic distribution sort of appraoch similar to the suggestions given by Joscha Bach in the Generalist AI talk between Joscha Bach and Michael Levin. We will attempt to track how much each neuron contributes to the final reward.
* Input neurons would be rewarded more for keeping the input information to encourage the system to keep the information.
* A basic training test we can imagine is a simple addition algorithm. For instance, we can create a simple 4 bit adder. To do so, we will have 2 sets of 4 cell inputs, and one set of a 4 bit output with a carry output.
* If a cell is negatively contributing to the loss function (or whatever metric we use to evaluate the result), cells will weaken their connection to it and eventually seek new neighbors, and if they're positively contributing cells will strengthen their connection. This can draw on the cells memory in some fashion as well.
* If a cell is itself negatively contributing to the loss function, it will randomly seek new neighbors in a hope to eventually contribute.
* Unlike a traditional neural net which would be trained and then ran in inference, the training of this automata would be continuously ongoing. I'll need to add ways to interact with it live to help test how resistant it is to perturbations over time and learning entirely new tasks within the network, for isntance starting with a 2 bit adder, upping it to 4, upping that to 8, etc.
* Since information would take time to propagate through the network and calculate (unless there was somehow direct connections to the task, but in practice the tasks should be too complex for such simple connections), the method by which rewards are propagated through the network should probably be analyzed every x steps or something of the sort. The memory could in theory play some sort of part in this, and in general it seems to line up with my recent thoughts on the importance of brain-wave like patterns in intelligence in general. This could in practice mean we can give a new input and read the output every x steps, say 10, making the network have an effective processing time of x steps * time per step.